In [1]:
## 위클리 스타일링의 누끼이미지(제품, 썸네일)를 적용합니다.
# 최종누끼이미지를 finish.zip로 압축합니다.
# 완료이미지 폴더를 더블클릭합니다. 
# 완료이미지 폴더 내에 최종누끼이미지파일을 압축한 finish.zip파일을 드레그하여 이동시켜줍니다.
# 실행파일로 돌아와 실행(▶ Run)합니다.

In [2]:
import os
import zipfile
import sys

sys.path.append('/home/lib')
config = __import__('config')

# 압축 파일 경로
zip_path = "./완료이미지/finish.zip"

# 압축 파일 열기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # 압축 해제
    zip_ref.extractall('./완료이미지')
    print("압축 해제 완료")

ModuleNotFoundError: No module named 'config'

In [ ]:
# 교체할 이미지 파일 수집
try:
    if os.path.exists('products.txt'):
        os.remove('products.txt')
except Exception as e:
    pass

os.system(f'find ./완료이미지 -not -path "./완료이미지/__MACOSX/*" -name "*_1*" >> ./완료이미지/products.txt')

exp_list = ['.png', '.jpeg', '.PNG', '.JPEG', '.JPG']

suggestion = set()
suggestion_product = set()

with open("./완료이미지/products.txt", "r") as f:
    products = f.readlines()
    product_list = []
    for product in products:
        product = product.replace('\n', '')
        new_product = product
        pid = product.split('/')[-1].split('_')[0]
        suggestion.add(product.split('/')[3])
        suggestion_product.add(pid)
        for exp in exp_list:
            new_product = new_product.replace(exp, '.jpg')
                
        if product != new_product:
            os.system(f'mv {product} {new_product}')

        product_list.append(int(pid))


# S3에 접근하여 이미지 교체
os.system(
        'aws s3 cp ./완료이미지/finish s3://stylerecipe-korea/media/suggestion/ --recursive --acl public-read')


In [ ]:
# 교체된 파일을 기준으로 데이터베이스 썸네일교체, 착용샷 교체 작업
cur = config.conn.cursor()

updateS = tuple(suggestion)
updateSP = tuple(suggestion_product)

query = cur.execute("""
update "Suggestion_suggestion" ss 
set thumbnail = concat('suggestion/',ss.id::varchar,'/',ss.id::varchar,'.jpg') 
where ss.id in %s
""", (updateS,))

print("썸네일 변경 완료 : ", updateS)

query2 = cur.execute("""
update "Suggestion_product" sp
set wearing_image =  concat('suggestion/',sp.suggestion_id::varchar,'/',sp.id::varchar,'_1.jpg')
where id in %s
""", (updateSP,))

print("착용샷 변경 완료 : ", updateSP)

config.conn.commit()
config.conn.close()


In [ ]:
os.system(f'rm -rf ./완료이미지/*')
print("종료")